# 1. Imports

In [2]:
import pandas as pd
import numpy as np

# 2. Data Preprocessing

In [3]:
def convert_to_category(X):
    for col in X.columns:
        if ( (X[col].dtype != 'int64') and (X[col].dtype != 'float64') and (X[col].dtype != 'bool')):
            X[col] = X[col].astype('category')
            
    return X

In [6]:
stores_train = pd.read_csv('data/stores_train_with_extra_features.csv')
stores_test = pd.read_csv('data/stores_test_with_extra_features.csv')

plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')
grunnkrets_ages = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_household_types = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_household_income = pd.read_csv('data/grunnkrets_income_households.csv')
stores_extra = pd.read_csv('data/stores_extra.csv')
busstops = pd.read_csv('data/busstops_Norway.csv')


stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns='sales_channel_name', inplace=True)
plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')


grunnkrets_copy = grunnkrets.copy()
grunnkrets_copy.rename(columns={'year': 'year_1'}, inplace=True)

grunnkrets_ages_copy = grunnkrets_ages.copy()
grunnkrets_ages_copy.rename(columns={'year': 'year_2'}, inplace=True)
grunnkrets_ages_copy['grunnkrets_population'] = grunnkrets_ages_copy.iloc[:, 2:].sum(axis=1)

grunnkrets_household_types_copy = grunnkrets_household_types.copy()
grunnkrets_household_types_copy.rename(columns={'year': 'year_3'}, inplace=True)
grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]] = grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]].astype('int64')

grunnkrets_household_income_copy = grunnkrets_household_income.copy()
grunnkrets_household_income_copy.rename(columns={'year': 'year_4', 'singles': 'singles_income','couple_without_children':'couple_without_children_income'}, inplace=True)
grunnkrets_household_income_copy.rename(columns={'singles': 'singles_income', }, inplace=True)

    

df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')



df['grunnkrets_population_density'] = df['grunnkrets_population'] / df['area_km2']


df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)



columns_to_drop = [
                  'store_id',
                  'plaace_hierarchy_id',
                  'grunnkrets_id',
                  'year',
                  'address',
                  'store_name',
                  'year_1',
                  'geometry',
                  'grunnkrets_name',
                  'district_name',
                  'municipality_name',
                  'year_2',
                  'year_3',
                  'year_4',
                  'sales_channel_name',
                  'lv1',
                  'lv2',
                  'lv3',
                  'lv4',
                  ]


X = df.drop(columns=columns_to_drop)
X = convert_to_category(X)

X['revenue'] = np.log1p(X['revenue'])


y = df.revenue
y = np.log1p(y)


categorical_features = X.select_dtypes(include=['category']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns


X[numerical_columns] = np.log(X[numerical_columns])
X['revenue'] = np.exp(X['revenue'])

c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [4]:
stores_test_copy = stores_test.copy()

df = stores_test_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')

df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)
df['grunnkrets_population_density'] = df['grunnkrets_population'] / df['area_km2']

X_test = df.drop(columns = columns_to_drop)
X_test = convert_to_category(X_test)


numerical_columns = X_test.select_dtypes(include=['int64', 'float64']).columns

X_test[numerical_columns] = np.log(X_test[numerical_columns])

c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


# 3. Model

In [5]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,23 mins 46 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_levit_fgfmpc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.572 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [6]:
train = h2o.H2OFrame(X)

x = train.columns
y = "revenue"
x.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models=175, seed=1, sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train)

# 4. Predictions

In [ ]:
test = h2o.H2OFrame(X_test)

In [ ]:
y_pred = aml.leader.predict(test)
preds = np.expm1(y_pred.as_data_frame())

In [ ]:
submission = pd.DataFrame()
submission['id'] = stores_test['store_id']
submission['predicted'] = np.asarray(preds)

submission.to_csv('sample_submission_H20_175.csv', index=False)
submission

# 5. Feature Engineering

In [ ]:
oslo_lat = 59.9138688
oslo_lon = 10.7522454
bergen_lat = 60.391263
bergen_lon = 5.322054
trondheim_lat = 63.430515
trondheim_lon = 10.395053
stavanger_lat = 58.969976
stavanger_lon = 5.733107
drammen_lat = 59.742043
drammen_lon = 10.204421


from haversine import haversine, Unit

stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')

# calculate the distance to the cities

def distance_to_city(lat, lon, city_lat, city_lon):
    
    # set 
    distance = haversine((lat, lon), (city_lat, city_lon), unit=Unit.KILOMETERS)
        
    return distance

stores_train['distance_to_oslo'] = stores_train.apply(lambda x: distance_to_city(x['lat'], x['lon'], oslo_lat, oslo_lon), axis=1)
stores_test['distance_to_oslo'] = stores_test.apply(lambda x: distance_to_city(x['lat'], x['lon'], oslo_lat, oslo_lon), axis=1)

stores_train[['store_id', 'distance_to_oslo']].to_csv('data/other_data/stores_train_distance_to_oslo.csv', index=False)
stores_test[['store_id', 'distance_to_oslo']].to_csv('data/other_data/stores_test_distance_to_oslo.csv', index=False)

stores_train['distance_to_bergen'] = stores_train.apply(lambda x: distance_to_city(x['lat'], x['lon'], bergen_lat, bergen_lon), axis=1)
stores_test['distance_to_bergen'] = stores_test.apply(lambda x: distance_to_city(x['lat'], x['lon'], bergen_lat, bergen_lon), axis=1)

stores_train[['store_id', 'distance_to_bergen']].to_csv('data/other_data/stores_train_distance_to_bergen.csv', index=False)
stores_test[['store_id', 'distance_to_bergen']].to_csv('data/other_data/stores_test_distance_to_bergen.csv', index=False)

# generate distances to Trondheim for stores_train and stores_test

stores_train['distance_to_trondheim'] = stores_train.apply(lambda x: distance_to_city(x['lat'], x['lon'], trondheim_lat, trondheim_lon), axis=1)
stores_test['distance_to_trondheim'] = stores_test.apply(lambda x: distance_to_city(x['lat'], x['lon'], trondheim_lat, trondheim_lon), axis=1)

stores_train[['store_id', 'distance_to_trondheim']].to_csv('data/other_data/stores_train_distance_to_trondheim.csv', index=False)
stores_test[['store_id', 'distance_to_trondheim']].to_csv('data/other_data/stores_test_distance_to_trondheim.csv', index=False)

stores_train['distance_to_stavanger'] = stores_train.apply(lambda x: distance_to_city(x['lat'], x['lon'], stavanger_lat, stavanger_lon), axis=1)
stores_test['distance_to_stavanger'] = stores_test.apply(lambda x: distance_to_city(x['lat'], x['lon'], stavanger_lat, stavanger_lon), axis=1)

stores_train[['store_id', 'distance_to_stavanger']].to_csv('data/other_data/stores_train_distance_to_stavanger.csv', index=False)
stores_test[['store_id', 'distance_to_stavanger']].to_csv('data/other_data/stores_test_distance_to_stavanger.csv', index=False)

stores_train['distance_to_drammen'] = stores_train.apply(lambda x: distance_to_city(x['lat'], x['lon'], drammen_lat, drammen_lon), axis=1)
stores_test['distance_to_drammen'] = stores_test.apply(lambda x: distance_to_city(x['lat'], x['lon'], drammen_lat, drammen_lon), axis=1)

stores_train[['store_id', 'distance_to_drammen']].to_csv('data/other_data/stores_train_distance_to_drammen.csv', index=False)
stores_test[['store_id', 'distance_to_drammen']].to_csv('data/other_data/stores_test_distance_to_drammen.csv', index = False)

In [ ]:
def find_shortest_distance_to_busstop(lat, lon):
    distances = busstops.apply(lambda x: haversine((lat, lon), (x['busstop_lat'], x['busstop_lon']), unit=Unit.METERS), axis=1)
    return distances.min()

def count_closest_busstops( lat, lon, radius):
    distances = busstops.apply(lambda x: haversine((lat, lon), (x['busstop_lat'], x['busstop_lon']), unit=Unit.METERS), axis=1)
    return len(distances[distances < radius] > 0)

stores_train['distance_to_closest_busstop'] = stores_train.apply(lambda x: find_shortest_distance_to_busstop(x['lat'], x['lon']), axis=1)
stores_test['distance_to_closest_busstop'] = stores_test.apply(lambda x: find_shortest_distance_to_busstop(x['lat'], x['lon']), axis=1)

stores_train[['store_id', 'distance_to_closest_busstop']].to_csv('data/other_data/stores_train_distance_to_closest_busstop.csv', index=False)
stores_test[['store_id', 'distance_to_closest_busstop']].to_csv('data/other_data/stores_test_distance_to_closest_busstop.csv', index=False)

radius_list = [100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, 15000, 20000]

for radius in radius_list:

    stores_train[f'num_closest_busstops_{radius}m'] = stores_train.apply(lambda x: count_closest_busstops(x['lat'], x['lon'], radius), axis=1)
    stores_test[f'num_closest_busstops_{radius}m'] = stores_test.apply(lambda x: count_closest_busstops(x['lat'], x['lon'], radius), axis=1)

    stores_train[['store_id', f'num_closest_busstops_{radius}m']].to_csv(f'stores_train_num_closest_busstops_{radius}m.csv', index=False)
    stores_test[['store_id', f'num_closest_busstops_{radius}m']].to_csv(f'stores_test_num_closest_busstops_{radius}m.csv', index=False)

In [ ]:
df = stores_train.copy()

df= pd.merge(df, plaace_hierarchy, how='left', on='plaace_hierarchy_id')

df.drop(columns=['revenue'], inplace=True)

stores_test = stores_test.copy()

stores_test = pd.merge(stores_test, plaace_hierarchy, how='left', on='plaace_hierarchy_id')

stores_extra = pd.merge(stores_extra, plaace_hierarchy, how='left', on='plaace_hierarchy_id')

df = pd.concat([stores_train, stores_test, stores_extra], ignore_index=True)


def find_closest_store(lat, lon, lv1, col):
    distances = df[df[col] == lv1].apply(lambda x: haversine((lat, lon), (x['lat'], x['lon']), unit=Unit.METERS), axis=1)
    # can't be the store it self
    return distances[distances > 0].min()

def count_closest_stores(lat, lon, lv, col, radius):
    distances = df[df[col] == lv].apply(lambda x: haversine((lat, lon), (x['lat'], x['lon']), unit=Unit.METERS), axis=1)
    return len(distances[distances < radius] > 0)

levels = ['lv1', 'lv2', 'lv3', 'lv4']
radius_list = [100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, 15000, 20000]

for level in levels:
    stores_train[f'closest_store_{level}'] = stores_train.apply(lambda x: find_closest_store(x['lat'], x['lon'], x[f'{level}_desc'], level), axis=1)
    stores_test[f'closest_store_{level}'] = stores_test.apply(lambda x: find_closest_store(x['lat'], x['lon'], x[f'{level}_desc'], level), axis=1)
    
    stores_train[['store_id', f'closest_store_{level}']].to_csv(f'stores_train_closest_store_{level}.csv', index=False)
    stores_test[['store_id', f'closest_store_{level}']].to_csv(f'stores_test_closest_store_{level}.csv', index=False)
    
    for radius in radius_list:
        stores_train[f'num_closest_stores_{level}_{radius}m'] = stores_train.apply(lambda x: count_closest_stores(x['lat'], x['lon'], x[f'{level}_desc'], level, radius), axis=1)
        stores_test[f'num_closest_stores_{level}_{radius}m'] = stores_test.apply(lambda x: count_closest_stores(x['lat'], x['lon'], x[f'{level}_desc'], level, radius), axis=1)

        stores_train[['store_id',f'num_closest_stores_lv1_{radius}m']].to_csv(f'stores_train_num_closest_stores_{level}_{radius}.csv', index=False)
        stores_test[['store_id',f'num_closest_stores_lv1_{radius}m']].to_csv(f'stores_test_num_closest_stores_{level}_{radius}m.csv', index=False)

In [ ]:
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')

# busstops
stores_train_closest_busstop = pd.read_csv('data/other_data/stores_train_closest_busstop.csv')
stores_test_closest_busstop = pd.read_csv('data/other_data/stores_test_closest_busstop.csv')

stores_train = pd.merge(stores_train, stores_train_closest_busstop, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_busstop, on='store_id', how='left')

stores_train_num_closest_busstop_100m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_100m.csv')
stores_test_num_closest_busstop_100m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_100m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_busstop_100m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_busstop_100m, on='store_id', how='left')

stores_train_num_closest_busstops_250m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_250m.csv')
stores_test_num_closest_busstops_250m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_250m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_busstops_250m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_busstops_250m, on='store_id', how='left')

stores_train_num_busstops_500m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_500m.csv')
stores_test_num_busstops_500m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_500m, on='store_id', how='left')

stores_train_num_busstops_750m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_750m.csv')
stores_test_num_busstops_750m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_750m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_750m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_750m, on='store_id', how='left')

stores_train_num_busstops_1000m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_1000m.csv')
stores_test_num_busstops_1000m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_1000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_1000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_1000m, on='store_id', how='left')

stores_train_num_busstops_2500m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_2500m.csv')
stores_test_num_busstops_2500m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_2500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_2500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_2500m, on='store_id', how='left')

stores_train_num_busstops_5000m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_5000m.csv')
stores_test_num_busstops_5000m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_5000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_5000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_5000m, on='store_id', how='left')

stores_train_num_busstops_7500m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_7500m.csv')
stores_test_num_busstops_7500m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_7500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_7500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_7500m, on='store_id', how='left')

stores_train_num_busstops_10000m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_10000m.csv')
stores_test_num_busstops_10000m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_10000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_10000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_10000m, on='store_id', how='left')

stores_train_num_busstops_12500m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_12500m.csv')
stores_test_num_busstops_12500m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_12500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_12500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_12500m, on='store_id', how='left')

stores_train_num_busstops_15000m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_15000m.csv')
stores_test_num_busstops_15000m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_15000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_15000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_15000m, on='store_id', how='left')

stores_train_num_busstops_20000m = pd.read_csv('data/other_data/stores_train_num_closest_busstops_20000m.csv')
stores_test_num_busstops_20000m = pd.read_csv('data/other_data/stores_test_num_closest_busstops_20000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops_20000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops_20000m, on='store_id', how='left')


# stores

# closest store
stores_train_closest_store_lv1 = pd.read_csv('data/other_data/stores_train_closest_store_lv1.csv')
stores_test_closest_store_lv1 = pd.read_csv('data/other_data/stores_test_closest_store_lv1.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv1, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv1, on='store_id', how='left')

stores_train_closest_store_lv2 = pd.read_csv('data/other_data/stores_train_closest_store_lv2.csv')
stores_test_closest_store_lv2 = pd.read_csv('data/other_data/stores_test_closest_store_lv2.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv2, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv2, on='store_id', how='left')

stores_train_closest_store_lv3 = pd.read_csv('data/other_data/stores_train_closest_store_lv3.csv')
stores_test_closest_store_lv3 = pd.read_csv('data/other_data/stores_test_closest_store_lv3.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv3, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv3, on='store_id', how='left')

stores_train_closest_store_lv4 = pd.read_csv('data/other_data/stores_train_closest_store_lv4.csv')
stores_test_closest_store_lv4 = pd.read_csv('data/other_data/stores_test_closest_store_lv4.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv4, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv4, on='store_id', how='left')

# num closest stores

#lv1

stores_train_num_closest_stores_lv1_100m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_100m.csv')
stores_test_num_closest_stores_lv1_100m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_100m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_100m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_100m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_250m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_250m.csv')
stores_test_num_closest_stores_lv1_250m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_250m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_250m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_250m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_500m.csv')
stores_test_num_closest_stores_lv1_500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_500m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_750m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_750m.csv')
stores_test_num_closest_stores_lv1_750m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_750m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_750m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_750m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_1000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_1000m.csv')
stores_test_num_closest_stores_lv1_1000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_1000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_1000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_1000m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_2500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_2500m.csv')
stores_test_num_closest_stores_lv1_2500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_2500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_2500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_2500m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_5000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_5000m.csv')
stores_test_num_closest_stores_lv1_5000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_5000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_5000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_5000m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_7500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_7500m.csv')
stores_test_num_closest_stores_lv1_7500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_7500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_7500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_7500m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_10000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_10000m.csv')
stores_test_num_closest_stores_lv1_10000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_10000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_10000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_10000m, on='store_id', how='left')

stores_train_num_closest_stores_lv1_15000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1_15000m.csv')
stores_test_num_closest_stores_lv1_15000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1_15000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1_15000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1_15000m, on='store_id', how='left')


# lv2

stores_train_num_closest_stores_lv2_100m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_100m.csv')
stores_test_num_closest_stores_lv2_100m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_100m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_100m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_100m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_250m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_250m.csv')
stores_test_num_closest_stores_lv2_250m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_250m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_250m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_250m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_500m.csv')
stores_test_num_closest_stores_lv2_500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_500m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_750m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_750m.csv')
stores_test_num_closest_stores_lv2_750m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_750m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_750m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_750m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_1000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_1000m.csv')
stores_test_num_closest_stores_lv2_1000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_1000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_1000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_1000m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_2500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_2500m.csv')
stores_test_num_closest_stores_lv2_2500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_2500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_2500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_2500m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_5000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_5000m.csv')
stores_test_num_closest_stores_lv2_5000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_5000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_5000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_5000m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_7500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_7500m.csv')
stores_test_num_closest_stores_lv2_7500m= pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_7500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_7500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_7500m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_10000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_10000m.csv')
stores_test_num_closest_stores_lv2_10000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_10000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_10000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_10000m, on='store_id', how='left')

stores_train_num_closest_stores_lv2_15000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2_15000m.csv')
stores_test_num_closest_stores_lv2_15000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2_15000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2_15000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2_15000m, on='store_id', how='left')

# lv3

stores_train_num_closest_stores_lv3_100m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_100m.csv')
stores_test_num_closest_stores_lv3_100m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_100m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_100m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_100m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_250m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_250m.csv')
stores_test_num_closest_stores_lv3_250m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_250m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_250m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_250m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_500m.csv')
stores_test_num_closest_stores_lv3_500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_500m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_750m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_750m.csv')
stores_test_num_closest_stores_lv3_750m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_750m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_750m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_750m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_1000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_1000m.csv')
stores_test_num_closest_stores_lv3_1000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_1000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_1000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_1000m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_2500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_2500m.csv')
stores_test_num_closest_stores_lv3_2500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_2500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_2500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_2500m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_5000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_5000m.csv')
stores_test_num_closest_stores_lv3_5000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_5000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_5000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_5000m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_7500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_7500m.csv')
stores_test_num_closest_stores_lv3_7500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_7500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_7500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_7500m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_10000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_10000m.csv')
stores_test_num_closest_stores_lv3_10000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_10000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_10000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_10000m, on='store_id', how='left')

stores_train_num_closest_stores_lv3_15000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3_15000m.csv')
stores_test_num_closest_stores_lv3_15000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3_15000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3_15000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3_15000m, on='store_id', how='left')


# lv4

stores_train_num_closest_stores_lv4_250m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_250m.csv')
stores_test_num_closest_stores_lv4_250m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_250m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_250m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_250m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_500m.csv')
stores_test_num_closest_stores_lv4_500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_500m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_750m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_750m.csv')
stores_test_num_closest_stores_lv4_750m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_750m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_750m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_750m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_1000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_1000m.csv')
stores_test_num_closest_stores_lv4_1000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_1000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_1000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_1000m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_2500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_2500m.csv')
stores_test_num_closest_stores_lv4_2500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_2500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_2500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_2500m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_5000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_5000m.csv')
stores_test_num_closest_stores_lv4_5000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_5000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_5000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_5000m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_7500m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_7500m.csv')
stores_test_num_closest_stores_lv4_7500m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_7500m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_7500m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_7500m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_10000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_10000m.csv')
stores_test_num_closest_stores_lv4_10000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_10000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_10000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_10000m, on='store_id', how='left')

stores_train_num_closest_stores_lv4_15000m = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4_15000m.csv')
stores_test_num_closest_stores_lv4_15000m = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4_15000m.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4_15000m, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4_15000m, on='store_id', how='left')

stores_train_distance_to_oslo = pd.read_csv('data/other_data/stores_train_distance_to_oslo.csv')
stores_test_distance_to_oslo = pd.read_csv('data/other_data/stores_test_distance_to_oslo.csv')

stores_train = pd.merge(stores_train, stores_train_distance_to_oslo, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_distance_to_oslo, on='store_id', how='left')

stores_train_distance_to_bergen = pd.read_csv('data/other_data/stores_train_distance_to_bergen.csv')
stores_test_distance_to_bergen = pd.read_csv('data/other_data/stores_test_distance_to_bergen.csv')

stores_train = pd.merge(stores_train, stores_train_distance_to_bergen, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_distance_to_bergen, on='store_id', how='left')

stores_train_distance_to_trondheim = pd.read_csv('data/other_data/stores_train_distance_to_trondheim.csv')
stores_test_distance_to_trondheim = pd.read_csv('data/other_data/stores_test_distance_to_trondheim.csv')

stores_train = pd.merge(stores_train, stores_train_distance_to_trondheim, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_distance_to_trondheim, on='store_id', how='left')

stores_train_distance_to_stavanger = pd.read_csv('data/other_data/stores_train_distance_to_stavanger.csv')
stores_test_distance_to_stavanger = pd.read_csv('data/other_data/stores_test_distance_to_stavanger.csv')

stores_train = pd.merge(stores_train, stores_train_distance_to_stavanger, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_distance_to_stavanger, on='store_id', how='left')

stores_train_distance_to_drammen = pd.read_csv('data/other_data/stores_train_distance_to_drammen.csv')
stores_test_distance_to_drammen = pd.read_csv('data/other_data/stores_test_distance_to_drammen.csv')

stores_train = pd.merge(stores_train, stores_train_distance_to_drammen, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_distance_to_drammen, on='store_id', how='left')

stores_train.to_csv('data/stores_train_with_extra_features.csv', index=False)
stores_test.to_csv('data/stores_test_with_extra_features.csv', index=False)